In [5]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv3D, Conv2DTranspose, Conv3DTranspose, Dense, Reshape, Flatten, concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import tensorflow as tf


In [80]:

file_path = "../CSV/data_rad_clin_DEF.csv"

data = pd.read_csv(file_path)
labels_column = data['label']
labels = labels_column.astype(int).tolist()

labels=np.array(labels)

# Estrazione dei numeri dai nomi dei pazienti
loaded_patients = data['IDs_new'].str.extract(r'(\d+)').astype(int).squeeze().tolist()

print("Labels:", labels)
print("Number of labels:", len(labels))
print("Patient Names: ", loaded_patients )

# Estrazione immagini
h5_file_path = '../images_by_patient_final.h5'

loaded_class_images = []

with h5py.File(h5_file_path, 'r') as h5_file:
    for key in h5_file.keys():
        images = np.array(h5_file[key])
        loaded_class_images.append(images)


print("Lunghezza della lista dei batch di immagini:", len(loaded_class_images))
print("Shape del batch di immagini del primo paziente:", loaded_class_images[0].shape)
print("Shape del batch di immagini del secondo paziente:", loaded_class_images[1].shape)



Labels: [0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 1 1 0 1 1 0 1 0 0 0 1 1 1 0 1 0
 0 0 1 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 1 0 0
 0 1 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0
 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1]
Number of labels: 129
Patient Names:  [5, 12, 15, 16, 17, 19, 22, 23, 24, 25, 26, 27, 29, 30, 31, 33, 35, 36, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 50, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 64, 65, 68, 69, 70, 71, 74, 75, 76, 78, 79, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 123, 124, 126, 127, 128, 129, 133, 135, 136, 137, 138, 139, 141, 142, 144, 146, 147, 149, 150, 153, 155, 158, 159, 161, 163, 166, 168, 169, 170, 171, 175, 176, 178, 182, 183, 188, 189, 190, 193, 197, 199, 200, 205]
Lunghezza della lista dei batch di immagini: 129
Shape del batch di immagini del primo paziente: (28, 64, 64)
S

# Tensorflow

In [88]:

def pad_or_trim_volumes(volumes, target_slices=64):
    """
    Uniforma tutti i volumi a 64 slice.
    Se il volume ha meno di 64 slice, aggiunge padding con slice nere.
    Se il volume ha più di 64 slice, rimuove le prime e ultime slice.

    Args:
        volumes (list of np.ndarray): lista di volumi 3D (ogni volume ha shape [n_slices, height, width]).
        target_slices (int): numero di slice desiderato, di default 64.

    Returns:
        padded_volumes (np.ndarray): volumi uniformati con 64 slice.
    """
    padded_volumes = []
    
    for volume in volumes:
        num_slices, height, width = volume.shape
        
        if num_slices < target_slices:
            padding_slices = target_slices - num_slices
            padding = np.zeros((padding_slices, height, width), dtype=volume.dtype)
            padded_volume = np.concatenate((volume, padding), axis=0)
        
        elif num_slices > target_slices:
   
            excess_slices = num_slices - target_slices
   
            start_trim = excess_slices // 2
            end_trim = excess_slices - start_trim
            padded_volume = volume[start_trim:num_slices-end_trim]
        
        else:

            padded_volume = volume
        
        padded_volumes.append(padded_volume)
    
    padded_volumes = np.array(padded_volumes, dtype=np.float32)
    padded_volumes = np.expand_dims(padded_volumes, axis=-1) 
    
    return padded_volumes



In [89]:
padded_volumes=pad_or_trim_volumes(loaded_class_images)

In [92]:


def build_encoder(latent_dim=128):
    """
    Crea un encoder che usa convoluzioni 2D per ciascuna slice 2D.
    """
    input_slice = Input(shape=(64, 64, 1))  # Ogni slice è di dimensioni (64, 64, 1)
    
    # Convoluzioni 2D per estrarre le feature da ciascuna slice
    x = Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu')(input_slice)
    x = Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    
    # Appiattisci e riduci le feature a un vettore latente
    x = Flatten()(x)
    latent = Dense(latent_dim, activation='relu')(x)
    
    encoder_model = Model(inputs=input_slice, outputs=latent)
    return encoder_model

def build_decoder(latent_dim=128):
    """
    Crea un decoder che ricostruisce il volume 3D dalle rappresentazioni latenti.
    """
    latent_input = Input(shape=(latent_dim,))
    
    # Decoder per riportare il volume alle dimensioni originali
    x = Dense(512 * 4 * 4 * 4, activation='relu')(latent_input)
    x = Reshape((4, 4, 4, 512))(x)

    # Strati di deconvoluzione 3D per ricostruire il volume
    x = Conv3DTranspose(256, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = Conv3DTranspose(128, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = Conv3DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = Conv3DTranspose(32, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = Conv3DTranspose(1, kernel_size=3, padding='same', activation='sigmoid')(x)  # Dimensione (64, 64, 64, 1)
    
    decoder_model = Model(inputs=latent_input, outputs=x)
    return decoder_model

from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K

def build_autoencoder(latent_dim=128, n_slices=64):
    """
    Costruisce l'autoencoder combinando l'encoder 2D per ciascuna slice e il decoder 3D.
    """
    input_volume = Input(shape=(n_slices, 64, 64, 1))  # Volume 3D con 64 slice 2D (64x64)

    # Encoder per processare ciascuna slice individualmente
    encoder_2d = build_encoder(latent_dim)
    
    # Itera sulle slice e applica l'encoder a ciascuna
    encoded_slices = []
    for i in range(n_slices):
        slice_2d = Lambda(lambda x: x[:, i, :, :, :])(input_volume)  # Estrae la i-esima slice 2D
        encoded_slice = encoder_2d(slice_2d)  # Applica l'encoder alla slice
        encoded_slices.append(encoded_slice)

    # Usa un Lambda layer per stack e combinare le rappresentazioni latenti
    encoded_volume = Lambda(lambda x: K.stack(x, axis=1))(encoded_slices)
    latent_vector = Lambda(lambda x: K.mean(x, axis=1))(encoded_volume)  # Media delle rappresentazioni latenti

    # Decoder per ricostruire il volume
    decoder_3d = build_decoder(latent_dim)
    reconstructed_volume = decoder_3d(latent_vector)
    
    # Modello completo autoencoder
    autoencoder = Model(inputs=input_volume, outputs=reconstructed_volume)
    return autoencoder

# Costruisci il modello autoencoder
autoencoder = build_autoencoder(latent_dim=128, n_slices=64)

# Compila il modello
autoencoder.compile(optimizer='adam', loss='mse')

# Mostra il sommario del modello
autoencoder.summary()


Model: "functional_35"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_27      │ (None, 64, 64,    │          0 │ -                 │
│ (InputLayer)        │ 64, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_297 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_298 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_299 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_300 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_301 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_302 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_303 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_304 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_305 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_306 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_307 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_308 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_309 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_310 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_311 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_312 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_313 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_314 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_315 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_316 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_317 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_318 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_319 (Lambda) │ (None, 64, 64, 1) │          0 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 10,069,953 (38.41 MB)

 Trainable params: 10,069,953 (38.41 MB)

 Non-trainable params: 0 (0.00 B)

In [93]:
def train_autoencoder(autoencoder, padded_volumes, epochs=20, batch_size=4):
    """
    Funzione per allenare l'autoencoder con i dati forniti.
    
    Args:
        autoencoder: il modello autoencoder.
        padded_volumes: array dei volumi con padding (batch, 97, 64, 64, 1).
        epochs: numero di epoche di allenamento.
        batch_size: dimensione del batch.
    """
    # Creazione del dataset da TensorFlow
    dataset = tf.data.Dataset.from_tensor_slices((padded_volumes, padded_volumes))
    dataset = dataset.batch(batch_size)
    
    # Esegui l'allenamento
    autoencoder.fit(dataset, epochs=epochs)

# Esegui l'allenamento
train_autoencoder(autoencoder, padded_volumes, epochs=20, batch_size=4)


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 995ms/step - loss: 833.7526
Epoch 2/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - loss: 829.9172
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 31s 924ms/step - loss: 829.6662
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - loss: 829.6247
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - loss: 829.6149
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - loss: 829.6042
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - loss: 829.5778
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - loss: 829.4620
Epoch 9/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - loss: 829.4028
Epoch 10/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - loss: 829.3747
Epoch 11/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 33s 991ms/step - loss: 829.3672
Epoch 12/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 30s 922ms/step - loss: 829.3633
Epoch 13/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - loss: 829.3652
Epoch 14/20
17/33 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - loss: 861.2743

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def visualize_reconstruction(original_volume, reconstructed_volume, slice_idx=None):
    """
    Funzione per visualizzare la slice centrale di un volume originale e ricostruito.
    
    Args:
        original_volume: volume originale.
        reconstructed_volume: volume ricostruito dal modello.
        slice_idx: indice della slice da visualizzare.
    """
    if slice_idx is None:
        slice_idx = original_volume.shape[1] // 2  # Slice centrale

    plt.figure(figsize=(10, 5))

    # Visualizza il volume originale
    plt.subplot(1, 2, 1)
    plt.imshow(original_volume[0, slice_idx, :, :, 0], cmap='gray')
    plt.title('Originale')

    # Visualizza il volume ricostruito
    plt.subplot(1, 2, 2)
    plt.imshow(reconstructed_volume[0, slice_idx, :, :, 0], cmap='gray')
    plt.title('Ricostruzione')

    plt.show()

some_test_volume = np.expand_dims(padded_volumes[0], axis=-1)  # Esempio di volume test
reconstructed_volume = autoencoder.predict(np.expand_dims(some_test_volume, axis=0))

# Visualizza la ricostruzione
visualize_reconstruction(some_test_volume, reconstructed_volume)
